In [18]:
import cv2
import numpy as np
import math
############################
image_path="image/Lake/4.2.06.tiff";
#Try with grayscale first#
img=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE);
#Or compress each channel one by one#
#img=cv2.imread(image_path);
###############Encoding##################
bitstream_output=EncodingImage(img);
file1=open("image_runbit.txt","w");
file1.write(bitstream_output);
file1.close();

############Decoding#####################
with open("image_runbit.txt","r") as myfile:
    runbits=myfile.read();
compressed_img=DecodingImage(runbits);
cv2.imwrite("compressed_image.bmp",np.uint8(compressed_img))

True

In [17]:
import numpy as np

def RunLengthEncoding(image):
    i=0;
    skip=0;
    stream=[];
    bitstream="";
    image=image.astype('int16');
    while i<image.shape[0]:
        if image[i] != 0:
            stream.append((image[i],skip));
            bitstream = bitstream + str(image[i])+ " " +str(skip)+" ";
            skip = 0;
        else:
            skip=skip+1;
        i=i+1;
    return bitstream;


def EncodingImage(img,block_size=8):
    block_size=8;

    [H,W]=img.shape;
    num_h=math.ceil(H/block_size);
    num_h=np.int(num_h);
    num_w=math.ceil(W/block_size);
    num_w=np.int(num_w);

    h=block_size*num_h;
    w=block_size*num_w;

    extended_img=np.zeros((h,w),dtype='int32');
    extended_img[0:H,0:W]=img[0:H,0:W];

    quantization_mat= np.array([[16,11,10,16,24,40,51,61],[12,12,14,19,26,58,60,55],[14,13,16,24,40,57,69,56 ],[14,17,22,29,51,87,80,62],[18,22,37,56,68,109,103,77],[24,35,55,64,81,104,113,92],[49,64,78,87,103,121,120,101],[72,92,95,98,112,100,103,99]])

    #Save original images#
    cv2.imwrite('uncompressed.bmp',np.uint8(extended_img));
    for i in range(num_h):
        r_1=i*block_size
        r_2=r_1+block_size
        for j in range(num_w):
            c_1=j*block_size
            c_2=c_1+block_size;
            #################################
            #Extract block and DCT transform#
            #################################
            block=np.float32(extended_img[r_1:r_2,c_1:c_2]);
            DCT_block=cv2.dct(block);
            #DCT quantization#
            DCT_block_normalized=np.divide(DCT_block,quantization_mat)
            
            ######Zig zag to one dimension##########
            DCT_block_normalized_zigzag=zigzag(DCT_block_normalized)
            extended_img[r_1:r_2,c_1:c_2]=np.reshape(DCT_block_normalized_zigzag,(block_size,block_size));
    bitstream_output=RunLengthEncoding(extended_img.flatten());
    #Adding image information into the bit stream#
    bitstream_output=str(extended_img.shape[0]) + " " + str(extended_img.shape[1]) + " "+ bitstream_output +";"
    return bitstream_output;

####################Decoding########################
def DecodingImage(runbits,block_size=8):
    quantization_mat = np.array([[16,11,10,16,24,40,51,61],[12,12,14,19,26,58,60,55],[14,13,16,24,40,57,69,56 ],[14,17,22,29,51,87,80,62],[18,22,37,56,68,109,103,77],[24,35,55,64,81,104,113,92],[49,64,78,87,103,121,120,101],[72,92,95,98,112,100,103,99]])
    details=runbits.split();
    h = int(''.join(filter(str.isdigit, details[0])))
    w = int(''.join(filter(str.isdigit, details[1])))
    output_mat=np.zeros(h*w).astype('int32');
    k=0;
    i=2;
    x=0;
    j=0;
    while k < output_mat.shape[0]:
        if(details[i]==';'):
            break
        if "-" not in details[i]:
            output_mat[k] = int(''.join(filter(str.isdigit, details[i])))        
        else:
            output_mat[k] = -1*int(''.join(filter(str.isdigit, details[i])))        
        if(i+3 < len(details)):
            j = int(''.join(filter(str.isdigit, details[i+3])))
        if j == 0:
            k=k+1
        else:                
            k=k+j+1        
        i=i+2
    output_mat=np.reshape(output_mat,(h,w));
    i=0;
    j=0;
    k=0;
    extended_img=np.zeros((h,w),dtype='int32');
    while i<h:
        j=0;
        while j<w:
            temp_stream=output_mat[i:i+8,j:j+8];
            block=inverse_zigzag(temp_stream.flatten(),int(block_size),int(block_size));
            block_dequantized=np.multiply(block,quantization_mat)
            extended_img[i:i+block_size,j:j+block_size]=cv2.idct(block_dequantized);
            j=j+block_size;
        i=i+block_size;
        
    extended_img[extended_img>255]=255;
    extended_img[extended_img<0]=0;
    return extended_img;
            

    




#########################Zigzag function get from https://github.com/amzhang1/simple-JPEG-compression########################

def zigzag(input):
    #initializing the variables
    #----------------------------------
    h = 0
    v = 0

    vmin = 0
    hmin = 0

    vmax = input.shape[0]
    hmax = input.shape[1]
    
    #print(vmax ,hmax )

    i = 0

    output = np.zeros(( vmax * hmax))
    #----------------------------------

    while ((v < vmax) and (h < hmax)):
    	
        if ((h + v) % 2) == 0:                 # going up
            
            if (v == vmin):
            	#print(1)
                output[i] = input[v, h]        # if we got to the first line

                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1                        

                i = i + 1

            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
            	#print(2)
            	output[i] = input[v, h] 
            	v = v + 1
            	i = i + 1

            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
            	#print(3)
            	output[i] = input[v, h] 
            	v = v - 1
            	h = h + 1
            	i = i + 1

        
        else:                                    # going down

        	if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
        		#print(4)
        		output[i] = input[v, h] 
        		h = h + 1
        		i = i + 1
        
        	elif (h == hmin):                  # if we got to the first column
        		#print(5)
        		output[i] = input[v, h] 

        		if (v == vmax -1):
        			h = h + 1
        		else:
        			v = v + 1

        		i = i + 1

        	elif ((v < vmax -1) and (h > hmin)):     # all other cases
        		#print(6)
        		output[i] = input[v, h] 
        		v = v + 1
        		h = h - 1
        		i = i + 1




        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
        	#print(7)        	
        	output[i] = input[v, h] 
        	break

    #print ('v:',v,', h:',h,', i:',i)
    return output




# Inverse zigzag scan of a matrix
# Arguments are: a 1-by-m*n array, 
# where m & n are vertical & horizontal sizes of an output matrix.
# Function returns a two-dimensional matrix of defined sizes,
# consisting of input array items gathered by a zigzag method.
#
# Matlab Code:
# Alexey S. Sokolov a.k.a. nICKEL, Moscow, Russia
# June 2007
# alex.nickel@gmail.com


def inverse_zigzag(input, vmax, hmax):
	
	#print input.shape

	# initializing the variables
	#----------------------------------
	h = 0
	v = 0

	vmin = 0
	hmin = 0

	output = np.zeros((vmax, hmax))

	i = 0
    #----------------------------------

	while ((v < vmax) and (h < hmax)): 
		#print ('v:',v,', h:',h,', i:',i)   	
		if ((h + v) % 2) == 0:                 # going up
            
			if (v == vmin):
				#print(1)
				
				output[v, h] = input[i]        # if we got to the first line

				if (h == hmax):
					v = v + 1
				else:
					h = h + 1                        

				i = i + 1

			elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
				#print(2)
				output[v, h] = input[i] 
				v = v + 1
				i = i + 1

			elif ((v > vmin) and (h < hmax -1 )):    # all other cases
				#print(3)
				output[v, h] = input[i] 
				v = v - 1
				h = h + 1
				i = i + 1

        
		else:                                    # going down

			if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
				#print(4)
				output[v, h] = input[i] 
				h = h + 1
				i = i + 1
        
			elif (h == hmin):                  # if we got to the first column
				#print(5)
				output[v, h] = input[i] 
				if (v == vmax -1):
					h = h + 1
				else:
					v = v + 1
				i = i + 1
        		        		
			elif((v < vmax -1) and (h > hmin)):     # all other cases
				output[v, h] = input[i] 
				v = v + 1
				h = h - 1
				i = i + 1




		if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
			#print(7)        	
			output[v, h] = input[i] 
			break


	return output





